### Funtions

In [150]:
def limpieza_estado(df, col):
    new = "inicio"
    for i in range(df.shape[0]):
        if (pd.isna(df.loc[i, col])):
            df.loc[i, col] = new
        else:
            new = df.loc[i, col]
    return df 

In [2]:
def limpiar_archivo(path_read, col, clas, columns, path_write):
    df = pd.read_csv(path_read, names=columns, skiprows=1)
    df = df[~(df[col[0]].isna() & df[col[1]].isna())].reset_index().drop(columns={"index"})
    
    # limpiamos una columna
    df = limpieza_estado(df, col[0])
    
    # imputamos la segunda columna
    df[col[1]].fillna("Total Estado", inplace=True)
    
    # creamos una columna de clasificacion
    df["clas_visitantes"] = df[col[1]].apply(lambda x: x if x in clas else "Total Visitantes")
    
    ### Imputamos la columna museo
    df[col[1]] = df[col[1]].apply(lambda x: x if not(x in clas) else np.nan)

    df = limpieza_estado(df, col[1])
    
    df.to_csv(path_write, index=False)
    print("Listo!!!")

### Modificación del archivo ciudades.txt

In [3]:
import pandas as pd 
import numpy as np

In [20]:
ciudades = pd.read_table("../data/ciudades.txt", names=["ciudades"])
ciudades = ciudades.reset_index()["ciudades"].reset_index()
ciudades["ciudades"] = ciudades.ciudades.apply(lambda x: x.split(",")[0])

ciudades = ciudades["ciudades"].reset_index()
ciudades.to_csv("../data/ciudades.csv", index=False)

# Creación de las variables para todas las ciudades

## Museos

In [151]:
names_columns_museo = ['estado', 'museo', '2000', '2001', '2002', '2003', '2004', '2005',
       '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013',
       '2014', '2015', '2016', '2017', '2018', '2019']

path_read_museo = "../data/raw/personas_museos_anual.csv"
path_write_museo = "../data/processed/estado_museo_clas_anual.csv"
col_museo = ["estado", "museo"]
clas_museo = ['    Visitantes Nacionales', '    Visitantes Internacionales']


In [152]:
limpiar_archivo(path_read_museo, col_museo, clas_museo, names_columns_museo, path_write_museo)

Listo!!!


### Zonas arqueologicas

In [153]:
names_columns_za = ['estado', 'zona_arqueologica', '2000', '2001', '2002', '2003', '2004',
       '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013',
       '2014', '2015', '2016', '2017', '2018', '20191']
path_read_za = "../data/raw/personas_zonas_arqueologicas_anual.csv"
path_write_za = "../data/processed/estado_zonas_arqueologicas_clas_anual.csv"
col_za = ["estado", "zona_arqueologica"]
clas_za = ['    Visitantes Nacionales','    Visitantes Internacionales']

In [154]:
limpiar_archivo(path_read_za, col_za, clas_za, names_columns_za, path_write_za)

Listo!!!


### Aeropuerto

In [155]:
# cargamos los datos
vuelos = pd.read_csv("../data/raw/vuelos_anuales.csv")

# clase de vuelos
clas_vuelos = ['   Nacionales', '   Internacionales',
       '   Regulares', '          Nacionales Regulares',
       '          Internacionales Regulares', '   Charters',
       '         Charters Nacionales',
       '         Charters Internacionales']
clas_vuelos_n = ['   Nacionales', '   Internacionales']

# filtramos la columna aeropuerto
vuelos["estado"] = vuelos.aeropuerto.apply(lambda x: x if not(x in clas_vuelos) else np.nan)
vuelos["vuelos"] = vuelos.aeropuerto.apply(lambda x: x if x in clas_vuelos_n else np.nan)

vuelos = vuelos[~(vuelos.estado.isna() & vuelos.vuelos.isna())].reset_index().drop(columns={"index"})

vuelos = limpieza_estado(vuelos, "estado")
vuelos.vuelos.fillna("Total Estado", inplace=True)
vuelos = vuelos[np.concatenate([['estado', 'vuelos'],vuelos.columns[1:-2]])]

In [156]:
vuelos.to_csv("../data/processed/vuelos_class_anual.csv", index=False)

### Hoteles

In [157]:
def var_hotel():
    df_final = pd.DataFrame([], columns=["centro_turístico", "categoria", "corredor_turístico"])
    for i in range(1,5):
        df = pd.read_excel("../data/5_2.xlsx", sheet_name="Vista0{}".format(str(i)), skiprows=18)
        if i==2:
            df.drop(columns={'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3',"Unnamed: 0", "Unnamed: 10"}, inplace=True)
        else:
            df.drop(columns={'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3',"Unnamed: 0"}, inplace=True)
        names_columns = df.loc[0].to_numpy()
        names_columns = [str.lower(i).replace("   ", "").replace(" ", "_") for i in names_columns]

        names_columns_new = []
        for i in names_columns:
            if i[-1]=="_":
                names_columns_new.append(i[0:-1])
            else:
                names_columns_new.append(i)

        df.columns = names_columns_new
        df = df.loc[1:].reset_index().drop(columns={"index"})
        df = limpieza_estado(df, "centro_turístico")
        df = df[~df.categoria.isna()]
        df["categoria"] = df.categoria.apply(lambda x: "{}_{}".format(str.lower(x.split()[1]), x.split()[0])) 
        
        categoria_hotel = ['estrella_1', 'estrellas_2', 'estrellas_3', 'estrellas_4',
       'estrellas_5']
        
        df = df[df.categoria.isin(categoria_hotel)]
        
        df_final = pd.merge(df_final, df, on=["centro_turístico", "categoria", "corredor_turístico"], how="outer")
        print("PASO!")
    return df_final

In [158]:
hotel = var_hotel()

PASO!
PASO!
PASO!
PASO!


In [159]:
hotel.to_csv("../data/processed/hotel_categoria_cuartos_turistas.csv", index=False)

## Origen destino

#### Pasajero

In [171]:
origen = pd.read_csv("../data/raw/origen_destino_pasajeros.csv", nrows=6828)

In [172]:
origen = limpieza_estado(origen, "Origen")
origen.Destino.fillna("Total", inplace=True)

In [173]:
origen.to_csv("../data/processed/origen_destino_pasajeros.csv", index=False)

#### Vuelos

In [167]:
origen = pd.read_csv("../data/raw/origen_destino_vuelos.csv")

In [168]:
origen = limpieza_estado(origen, "Origen")
origen.Destino.fillna("Total", inplace=True)

In [169]:
origen.head()

,Origen,Destino,1992,1993,1994,1995,1996,1997,1998,1999,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Acapulco,Total,6271,6795,8295,7039,6637,7378,7605,6477,...,4663,4019,3910,4227,4641,5087,4931,4366,4448,5592
1,Acapulco,Aguascalientes,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Acapulco,Cancún,17,17,10,17,17,37,34,11,...,0,0,1,0,0,1,0,0,0,28
3,Acapulco,Celaya,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Acapulco,Chihuahua,0,0,0,0,9,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [170]:
origen.to_csv("../data/processed/origen_destino_vuelos.csv", index=False)